In [ ]:
# ------------------------------------------------
# MessagesPlaceholder: đóng vai trò như bộ nhớ, duy trì input của người dùng cho generator
# ------------------------------------------------

import operator
from typing import TypedDict, List, Sequence, Annotated
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import MessageGraph, StateGraph, END

# Tạo prompt cho generate và reflection
generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "..."),
    MessagesPlaceholder(variable_name="messages"),
])

reflection_prompt = ...  # TODO: định nghĩa prompt reflection

# Dùng toán tử pipe (|) để kết hợp với LLM
generate_chain = generation_prompt | llm
reflect_chain = reflection_prompt | llm

# ------------------------------------------------
# MessageGraph: graph đặc biệt với state là mảng tin nhắn (HumanMessage, AIMessage, SystemMessage, ToolMessage)
# ------------------------------------------------

graph = MessageGraph()

# Định nghĩa state
class MessagesState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add] # We now track a list of messages

graph = StateGraph(MessagesState)

# ------------------------------------------------
# Xây dựng các node
# ------------------------------------------------

def generation_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    generated_tweet = generate_chain.invoke({"messages": state})
    # Khi node trả về AIMessage, LangGraph sẽ tự merge state bằng cơ chế tối ưu
    return [AIMessage(content=generated_tweet.content)]

def reflection_node(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    res = reflect_chain.invoke({"messages": state})
    return {"messages":[HumanMessage(content=res.content)]}

graph.add_node("generate", generation_node)
graph.add_node("reflect", reflection_node)

graph.add_edge("reflect", "generate")
graph.set_entry_point("generate")

# ------------------------------------------------
# Router node để kiểm soát luồng
# ------------------------------------------------

def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        return END
    return "reflect"

graph.add_conditional_edges("generate", should_continue)


Here are the flight options from Hanoi (HAN) to Ho Chi Minh City (SGN) on August 30, 2025:\n\n### Best Flight Options\n1. **Vietjet**\n   - **Departure:** 11:35 PM on Aug 30\n   - **Arrival:** 1:40 AM on Aug 31\n   - **Duration:** 2 hr 5 min\n   - **Stops:** Non-stop\n   - **Price:** ₫1,544,600\n\n2. **Vietravel Airlines**\n   - **Departure:** 9:05 PM on Aug 30\n   - **Arrival:** 11:20 PM on Aug 30\n   - **Duration:** 2 hr 15 min\n   - **Stops:** Non-stop\n   - **Price:** ₫1,624,840\n\n3. **Vietnam Airlines**\n   - **Departure:** 11:00 PM on Aug 30\n   - **Arrival:** 1:10 AM on Aug 31\n   - **Duration:** 2 hr 10 min\n   - **Stops:** Non-stop\n   - **Price:** ₫1,724,000\n\n4. **Vietnam Airlines**\n   - **Departure:** 11:15 PM on Aug 30\n   - **Arrival:** 1:25 AM on Aug 31\n   - **Duration:** 2 hr 10 min\n   - **Stops:** Non-stop\n   - **Price:** ₫1,724,000\n\n5. **Bamboo Airways**\n   - **Departure:** 1:35 PM on Aug 30\n   - **Arrival:** 3:45 PM on Aug 30\n   - **Duration:** 2 hr 10 min\n   - **Stops:** Non-stop\n   - **Price:** ₫1,750,000\n\n### Additional Options\n- **Vietjet**: \n  - Departure at 6:45 PM, Arrival at 8:55 PM, Price: ₫1,674,200\n  - Departure at 7:20 PM, Arrival at 9:30 PM, Price: ₫1,674,200\n- **Vietnam Airlines**: \n  - Departure at 9:00 PM, Arrival at 11:15 PM, Price: ₫1,951,000\n  - Departure at 9:40 PM, Arrival at 11:50 PM, Price: ₫1,951,000\n\n### Summary\n- The cheapest option is with **Vietjet** at ₫1,544,600 departing at 11:35 PM.\n- All options are non-stop flights, making them convenient for travel.\n\n### Recommendations\n- **Booking:** Consider booking soon as prices may increase closer to the date.\n- **Flexibility:** If your schedule allows, check for flights on nearby dates for potentially better prices.\n\nIf you need assistance with booking or further details, feel free to ask!
